In [0]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import current_timestamp, from_utc_timestamp


In [0]:
path_schema = '/Volumes/raw/tabnews/datalake/Ano_2022/Mês_05/Dia_06/Minuto_15/Segundo_20/*.json'
sample_df = spark.read.option('multiline', 'true').json(path_schema)

In [0]:
path = '/Volumes/raw/tabnews/datalake/Ano_*/Mês_*/Dia_*/Minuto_*/Segundo_*/*.json'
df = (spark.read.option('multiline', 'true')
                .schema(sample_df.schema)
                .json(path))
df_carga = df.withColumn('data_da_carga', from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"))
df_carga.createOrReplaceTempView('Raw_TabNews_LoadFull')

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW somente_ultima_versao AS
SELECT * 
FROM Raw_TabNews_LoadFull
QUALIFY ROW_NUMBER() OVER (PARTITION BY id ORDER BY UPDATED_AT DESC) = 1

In [0]:
df_somente_ultima_versao = spark.table("somente_ultima_versao")

df_somente_ultima_versao.write.mode('overwrite') \
                              .format('delta') \
                              .option('overwriteSchema', 'true') \
                              .option('mergeSchema', 'true') \
                              .saveAsTable('raw.tabnews.Raw_TabNews_LoadFull')